In [14]:
import os
from PIL import Image
from smolagents import CodeAgent, DuckDuckGoSearchTool, VisitWebpageTool, ToolCallingAgent
from smolagents import LiteLLMModel

In [1]:
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [ ]:
import os
from PIL import Image
from smolagents import CodeAgent, DuckDuckGoSearchTool, VisitWebpageTool

# model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
from smolagents import LiteLLMModel

# model = LiteLLMModel(
#         # model_id="ollama_chat/qwen2.5-coder:3b",  # Or try other Ollama-supported 
#         model_id="ollama_chat/gemma3:4b",
#         api_base="http://127.0.0.1:11434",  # Default Ollama local server
#         num_ctx=8192,
# )

model = LiteLLMModel(
    # model_name="gemini/gemini-2.0-flash",  # Use the litellm format for Gemini
    model_id="gemini/gemini-2.0-flash",
    # api_key=os.environ.get("GEMINI_API_KEY"),
    # provider="gemini",
    temperature=0.7,
    max_tokens=1024
)

task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe."""
# Also give me some supercar factories with the same cargo plane transfer time."""

agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)
agent.planning_interval = 4

result = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

In [ ]:
model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash",
    temperature=0,
    max_tokens=1024
)

web_agent = CodeAgent(
    model=model,
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)


In [45]:
find_city_coordinate_agent = CodeAgent(
    model=LiteLLMModel(
        model_id="gemini/gemini-2.0-flash", 
        temperature=0, 
        max_tokens=1024,
        # system_message="You are a helpful assistant that only responds with longitude and latitude coordinates when asked about cities."
        ),
    tools=[
        DuckDuckGoSearchTool(),
    ],
    name="find_city_coordinate_agent",
    description="Browses the web to find information about a city latitude and longitude coordinates",
    verbosity_level=1,
    max_steps=10,
    additional_authorized_imports=["json"],
)

# Example usage
find_city_coordinate_agent.run("Find the latitude and logitude of New York. Return only the coordinates.")

╭───────────────────────────────────── New run - find_city_coordinate_agent ──────────────────────────────────────╮
│                                                                                                                 │
│ Find the latitude and logitude of New York. Return only the coordinates.                                        │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = web_search(query="latitude and longitude of New York")                                          
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Latitude and longitude of New York City - Latlong.info](https://latlong.info/united-states/new-york/new-york-city)
The latitude of New York City, United States is 40.71427000, and the longitude is -74.00597000. New York City is 
located at United States country in the states place category with the gps coordinates of 40° 42' 51.372'' N and 
-74° 0' 21.492 E. Geographic coordinates are a way of specifying the location of a place on Earth, using a pair of 
...

[Where is New York City, NY, USA on Map Lat Long 
Coordinates](https://www.latlong.net/place/new-york-city-ny-usa-1848.html)
Find the latitude and longitude of New York City, NY, USA on a road map and a satellite map. Learn about the city's
history, attractions, and boroughs.

[Latitude and longitude of New York, United States - GPS Coordinates](https://latlong.info/united-states/new-york)
Find the latitude and longitude of New York, United States, in decimal degrees and DMS format, as well as UTM and 
GeoHash codes. See the map of New York and nearby cities with their coordinates.

[New York City Geographic coordinates - Latitude & longitude - 
Geodatos](https://www.geodatos.net/en/coordinates/united-states/new-york-city)
The latitude is the position relative to the equator, specifying the north-south position. The longitude specifies 
the east-west position measured from a reference meridian (usually the Greenwich Prime Meridian). The latitude and 
longitude of New York City have been calculated based on the geodetic datum WGS84. Map of New York City with 
coordinates

[Manhattan, New York City, NY, USA - 
LatLong](https://www.latlong.net/place/manhattan-new-york-city-ny-usa-31886.html)
The latitude of Manhattan, New York City, NY, USA is 40.776676, and the longitude is -73.971321. Manhattan, New 
York City, NY, USA is located at United States country in the Boroughs place category with the gps coordinates of 
40° 46' 36.0336'' N and 73° 58' 16.7556'' W.

[New York Latitude and Longitude Map - Maps of 
World](https://www.mapsofworld.com/usa/states/new-york/lat-long.html)
Latitude and longitude of New York are 40.7127° N, 74.0059° W. Use our New York lat long map to find coordinates 
anywhere in the state. Order Custom Maps Maps of the World Maps of the World ... New York Latitude and Longitude 
Map. by Vishul Malik April 3, 2025 4 Facebook Twitter Pinterest Email.

[Where is New York, USA on Map Lat Long Coordinates](https://www.latlong.net/place/new-york-usa-27719.html)
Find out the latitude and longitude of New York, USA, a state in the northeastern part of the country with many 
historic and cultural attractions. See the map, satellite view, and other states in the US.

[New York City Coordinates - Understanding NYC's Location](https://gottalovenewyork.com/new-york-city-coordinates/)
The coordinates for New York, New York are latitude 40.7128 and longitude -74.0060. What are New York Times Square 
coordinates? Times Square is at the cross section of 42nd Street, Seventh Avenue and Broadway, and stretches North 
to 47th Street. The coordinates for Times Square in New York City are latitude 40.7589 and longitude -73.9851.

[GPS coordinates of New York City, United States - 
Latitude.to](https://latitude.to/map/us/united-states/cities/new-york-city)
New York—often called New York City or the City of New York to distinguish it from the State of New York, of which 
it is a part—is the most populous city in the United States and the center of the New York metropolitan area, the 
premier gateway for legal immigration to the United States and one of the most populous urban agglomerations in the
world.

[Latitude and longitude of Manhattan - Latlong.info](https://latlong.info/united-states/new-york/manhattan)
The latitude of Manhattan, United States is 40.78343000, and the longitude is -73.96625000. Manhattan is located at
United States country in the states place category with the gps coordinates of 40° 47' 0.348'' N and -73° 57' 58.5 
E. 

[Step 1: Duration 2.74 seconds| Input tokens: 2,225 | Output tokens: 55]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("40.71427000, -74.00597000")                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 40.71427000, -74.00597000

[Step 2: Duration 1.42 seconds| Input tokens: 5,736 | Output tokens: 143]

'40.71427000, -74.00597000'

In [46]:
find_filming_location_agent = CodeAgent(
    model=LiteLLMModel(
        model_id="gemini/gemini-2.0-flash", 
        temperature=0, 
        max_tokens=1024,
        # system_message="You are a helpful assistant that provides a list of filming locations (cities only) for movies when requested, focusing solely on that information. Return only the list of filming locations in json format.",
        ),
    tools=[
        DuckDuckGoSearchTool(),
    ],
    name="find_filming_location_agent",
    description="Browses the web to find information about filming locations",
    verbosity_level=1,
    max_steps=5,
    additional_authorized_imports=["json"],
)

# Example usage
# find_filming_location_agent.run("Find the filming locations of the movie 'Batman'. Return only the list of filming locations.")
find_filming_location_agent.run("Find the filming locations of the movie 'Batman'. Return only the list of filming locations (city only) in json format.")

╭───────────────────────────────────── New run - find_filming_location_agent ─────────────────────────────────────╮
│                                                                                                                 │
│ Find the filming locations of the movie 'Batman'. Return only the list of filming locations (city only) in json │
│ format.                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import json                                                                                                      
                                                                                                                   
  search_results = web_search(query="filming locations of the movie Batman 1989")                                  
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Batman | 1989 - Worldwide Guide To Movie Locations](https://movie-locations.com/movies/b/Batman-1989.php)
Film locations for Tim Burton's Batman in London and Southeast England: Knebworth House, Knebworth, Hertfordshire 
('Wayne Manor' exterior); Hatfield House, Hatfield, Hertfordshire ('Wayne Manor' interior); Pinewood Studio, 
Buckinghamshire (studio); Ladies' Smoking Room, St Pancras Renaissance Hotel, Euston Road, NW1 (office of Gotham 
Globe); Little Barford Power Station, St Neots ...

[Batman (1989) - Filming & production - IMDb](https://m.imdb.com/title/tt0096895/locations/)
Batman: Directed by Tim Burton. With Michael Keaton, Jack Nicholson, Kim Basinger, Robert Wuhl. The Dark Knight of 
Gotham City begins his war on crime with his first major enemy being Jack Napier, a criminal who becomes the 
clownishly homicidal Joker.

[Where was the movie Batman filmed: | Filming locations, countries ...](https://wherefilmed.org/en/Batman1989/)
Gotham Globe on the map. Scene was shot St. Pancras Station was constructed by the Midland Railway in 1868 as the 
terminus of their only line into London. It was designed by William Henry Barlow with a single-span iron roof, the 
largest such roof at the time spanning 245 feet.

[80s Film Locations - Batman (1989) - Google 
Sites](https://sites.google.com/site/80sfilmlocations/Film-List/batman-1989)
Batman (1989) This big-budget adaptation of the popular DC Comics hero was directed by Tim Burton. Locations in 
England , primarily Pinewood Studios in Buckinghamshire, stood in for Gotham City.

[Where was Batman filmed? - Giggster](https://giggster.com/guide/movie-location/where-was-batman-filmed)
Batman (1989) is a groundbreaking superhero film directed by the talented Tim Burton. The title role of Batman, or 
his alter ego Bruce Wayne, is brought to life by Michael Keaton. Keaton's portrayal of the millionaire 
philanthropist turned vigilante crimefighter after witnessing his parents' brutal murder is compelling and 
relatable.

[Batman (1989) Filming Locations and Cast Details - 
filmiloc.com](https://www.filmiloc.com/batman-1989-filming-locations-and-cast-details/)
Batman, released in 1989, is an exciting mix of action and adventure. Directed by Tim Burton, the film tells the 
story of Gotham City's Dark Knight taking on his first major enemy, Jack Napier, who later becomes the notorious 
Joker. Its main stars include Michael Keaton as Batman, Jack Nicholson as Joker, and Kim Basinger portraying Vicki 
Vale.

[Batman (1989) Movie Filming Locations - The 80s Movies Rewind](https://fast-rewind.com/locations_batman.htm)
Batman Filming Locations. Part of 8 pages of info for the 1989 movie from the award winning '80s Movies Rewind. THE
WEB'S FAVORITE - SINCE 1999. Batman (1989) ... Rewind the locations where they filmed the 1989 Action / Adventure 
movie starring Michael Keaton, Jack Nicholson, Kim Basinger, ...

[theStudioTour.com - Archive - Batman](https://www.thestudiotour.com/archive/movie/batman-1989/)
Batman (1989) Directed by: Tim Burton. Cast: Michael Keaton, Jack Nicholson, Kim Basinger, Robert Wuhl, ... Filming
at Pinewood took place between October 1988 and January 1989. 18 soundstages were used, and almost all of the 95 
acres of available backlot. ... Additional locations included Knebworth House and Hatfield House doubling for Wayne
...

[Where was Batman filmed? | MovieLocate](https://movielocate.com/title/batman-1989/268/movie)
Filming Locations. Stevenage, Hertfordshire, United Kingdom London, Greater London, United Kingdom Iver, 
Buckinghamshire, United Kingdom Hatfield, Hertfordshire, United Kingdom Saint Neots, Bedford, United Kingdom. 
Filming Dates. 17 October 1988 - 12 January 1989. Studios. Be the first to add a studio

[Where they filmed Batman (1989) from The Movie Map film locations guide](http://www.themoviemap.com/Batman-1989)
This is where the movie Batman (1989) was filmed. The Movie Map - Find Film Locations Around The Wor

[Step 1: Duration 2.79 seconds| Input tokens: 2,235 | Output tokens: 90]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import json                                                                                                      
                                                                                                                   
  locations = []                                                                                                   
  locations.append("London")                                                                                       
  locations.append("Hatfield")                                                                                     
  locations.append("Stevenage")                                                                                    
  locations.append("Iver")                                                                                         
  locations.append("Saint Neots")                                                                                  
                                                                                                                   
  json_output = json.dumps(locations)                                                                              
  final_answer(json_output)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ["London", "Hatfield", "Stevenage", "Iver", "Saint Neots"]

[Step 2: Duration 1.39 seconds| Input tokens: 5,740 | Output tokens: 186]

'["London", "Hatfield", "Stevenage", "Iver", "Saint Neots"]'

In [47]:
manager_agent = CodeAgent(
    model=LiteLLMModel("gemini/gemini-2.0-flash", temperature=0, max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[find_city_coordinate_agent, find_filming_location_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "plotly.express",
        "shapely",
        "json",
        "pandas",
        "numpy",
        "json",
    ],
    planning_interval=5,
    verbosity_level=2,
    # final_answer_checks=[check_reasoning_and_plot],
    max_steps=10,
)

manager_agent.visualize()

CodeAgent | gemini/gemini-2.0-flash
├── ✅ Authorized imports: ['geopandas', 'plotly', 'plotly.express', 'shapely', 'json', 'pandas', 'numpy', 'json']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    ├── find_city_coordinate_agent | CodeAgent | gemini/gemini-2.0-flash
    │   ├── ✅ Authorized imports: ['json']
    │   ├── 📝 Description: Browses the web to find information about a city latitude and longitude coordinates
    │   └── 🛠️ Tools:
    │       ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    │       ┃ Name         ┃ Description                               ┃ Arguments                                ┃
    │       ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │       │ web_search   │ Performs a duckduckgo web search based on │ query (`string`): The search query to    │
    │       │              │ your query (think a Google search) then   │ perform.                                 │
    │       │              │ returns the top search results.           │                                          │
    │       │ final_answer │ Provides a final answer to the given      │ answer (`any`): The final answer to the  │
    │       │              │ problem.                                  │ problem                                  │
    │       └──────────────┴───────────────────────────────────────────┴──────────────────────────────────────────┘
    └── find_filming_location_agent | CodeAgent | gemini/gemini-2.0-flash
        ├── ✅ Authorized imports: ['json']
        ├── 📝 Description: Browses the web to find information about filming locations
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name         ┃ Description                               ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search   │ Performs a duckduckgo web search based on │ query (`string`): 

In [48]:
manager_agent.run("""
Find at most 5 Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W). Save the results as a pandas dataframe.
""")
#When calling the find_city_coordinate_agent, specifically ask to return only the longitude and latitude of a city.
# When calling the find_filming_location_agent, specifically ask to return only the list of filming locations (city only) at most 5. Example prompt: "Find the filming locations of the movie 'Batman'. Return only the list of filming locations (city only).".

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find at most 5 Batman filming locations in the world, calculate the time to transfer via cargo plane to here    │
│ (we're in Gotham, 40.7128° N, 74.0060° W). Save the results as a pandas dataframe.                              │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash ────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
*   Current location: Gotham (40.7128° N, 74.0060° W) - These are the coordinates of New York City, which is often 
used as a stand-in for Gotham.
*   Goal: Find at most 5 Batman filming locations.
*   Transportation: Cargo plane.

### 1.2. Facts to look up
*   Filming locations for Batman movies: Need to use `find_filming_location_agent` to find these.
*   Coordinates (latitude, longitude) for each filming location found: Need to use `find_city_coordinate_agent` for
each location if the filming location agent doesn't provide them.
*   Cruising speed of a typical cargo plane: While a default is provided in the `calculate_cargo_travel_time` tool,
it's good to confirm this value. Can be found via web search if needed.

### 1.3. Facts to derive
*   Distance between Gotham and each filming location. This will be calculated internally by 
`calculate_cargo_travel_time`.
*   Travel time from Gotham to each filming location using the cargo plane. This will be calculated by 
`calculate_cargo_travel_time`.
*   A pandas dataframe containing the filming locations and travel times.

## 2. Plan

1.  Use `find_filming_location_agent` to find up to 5 filming locations for Batman movies. The task should be 
phrased to return a list of locations.
2.  For each filming location returned by the agent:
    *   If the coordinates (latitude, longitude) are not provided by the `find_filming_location_agent`, use 
`find_city_coordinate_agent` to find the coordinates.
    *   Use `calculate_cargo_travel_time` to calculate the travel time from Gotham (40.7128° N, 74.0060° W) to the 
filming location.
3.  Store the filming locations and their corresponding travel times in a list of dictionaries.
4.  Convert the list of dictionaries into a pandas DataFrame.
5.  Return the pandas DataFrame using `final_answer`.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I will start by calling the `find_filming_location_agent` to find filming locations for Batman movies. I  
will ask for at most 5 locations.                                                                                  
Code:                                                                                                              
```py                                                                                                              
batman_filming_locations = find_filming_location_agent(task="Find at most 5 filming locations for Batman movies.") 
print(batman_filming_locations)                                                                                    
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_filming_locations = find_filming_location_agent(task="Find at most 5 filming locations for Batman         
  movies.")                                                                                                        
  print(batman_filming_locations)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I have the travel times to the filming locations. Now I will return the pandas DataFrame using the        
`final_answer` tool.                                                                                               
Code:                                                                                                              
```py                                                                                                              
final_answer(df)                                                                                                   
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(df)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer:            location  travel_time
0           Chicago         2.68
1            London         9.17
2         Hong Kong        20.00
3  Pinewood Studios         9.13

[Step 5: Duration 1.39 seconds| Input tokens: 21,621 | Output tokens: 691]

,location,travel_time
0,Chicago,2.68
1,London,9.17
2,Hong Kong,20.00
3,Pinewood Studios,9.13


In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
# from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = LiteLLMModel("gemini/gemini-2.0-flash", temperature=0, max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=LiteLLMModel("gemini/gemini-2.0-flash", temperature=0, max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "plotly.express",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

In [12]:
manager_agent.visualize()

CodeAgent | gemini/gemini-2.0-flash
├── ✅ Authorized imports: ['geopandas', 'plotly', 'plotly.express', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gemini/gemini-2.0-flash
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a duckduckgo web search  │ query (`string`): The search      │
            │                             │ based on your query (think a      │ query to perform.                 │
            │                             │ Google search) then returns the   │                                   │
            │                             │ top search results.               │                                   │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.                         │ destination_coords (`array`):     │
            │                             │     

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at most 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")